In [1]:
import numpy as np

In [2]:
import collections

In [3]:
import random

In [4]:
words = 'The sun is shining The weather is sweet The sun is shining the weather is sweet and one and one is two'.split()

In [5]:
vocabulary_size = 10

In [6]:
def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)  # 频率越高的词，在 dict 里面排的越前  
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)  # 最后返回的就是一个从 dict 获取的对 1700 万单词的编号
  count[0][1] = unk_count  # 修改 UNK 的值
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
# del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 1], ('is', 5), ('The', 3), ('weather', 2), ('sweet', 2)]
Sample data [2, 6, 1, 8, 2, 3, 1, 4, 2, 6] ['The', 'sun', 'is', 'shining', 'The', 'weather', 'is', 'sweet', 'The', 'sun']


In [7]:
count

[['UNK', 1],
 ('is', 5),
 ('The', 3),
 ('weather', 2),
 ('sweet', 2),
 ('one', 2),
 ('sun', 2),
 ('and', 2),
 ('shining', 2),
 ('the', 1)]

In [8]:
data

[2, 6, 1, 8, 2, 3, 1, 4, 2, 6, 1, 8, 9, 3, 1, 4, 7, 5, 7, 5, 1, 0]

In [9]:
batch_size=8
num_skips=2
skip_window=1

In [22]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    # _ 表示对循环中的实际值我们没有兴趣，为的是把 buffer 填充满,这里是 3
    for _ in range(span):  
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)  # 这个是过滤用的，如果没有，会出现指向重复，怎么实现过滤的？？
            print('target is {}, i * num_skips + j is {}, skip_windows is {},\nt2a is {}, '
                  'buffer[skip_window] is {}, buffer[target] is {}'.format(
                target, i* num_skips + j, skip_window, targets_to_avoid, buffer[skip_window], buffer[target]))
            print('---------------------------------------------')
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])  # 第二层循环完了刚好遍历完一个词，然后所有样本后读入下一个词
        data_index = (data_index + 1) % len(data) 
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], 
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

target is 2, i * num_skips + j is 0, skip_windows is 1,
t2a is [1, 2], buffer[skip_window] is 1, buffer[target] is 4
---------------------------------------------
target is 0, i * num_skips + j is 1, skip_windows is 1,
t2a is [1, 2, 0], buffer[skip_window] is 1, buffer[target] is 3
---------------------------------------------
target is 0, i * num_skips + j is 2, skip_windows is 1,
t2a is [1, 0], buffer[skip_window] is 4, buffer[target] is 1
---------------------------------------------
target is 2, i * num_skips + j is 3, skip_windows is 1,
t2a is [1, 0, 2], buffer[skip_window] is 4, buffer[target] is 2
---------------------------------------------
target is 2, i * num_skips + j is 4, skip_windows is 1,
t2a is [1, 2], buffer[skip_window] is 2, buffer[target] is 6
---------------------------------------------
target is 0, i * num_skips + j is 5, skip_windows is 1,
t2a is [1, 2, 0], buffer[skip_window] is 2, buffer[target] is 4
---------------------------------------------
target is 2, 

In [11]:
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  print(span)
  for _ in range(span):  # _ 表示对循环中的实际值我们没有兴趣，为的是把 buffer 填充满
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
#     print('buffer and data_index:', buffer, data_index)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
#       print('target and targets_to_avoid:', target, targets_to_avoid)
      batch[i * num_skips + j] = buffer[skip_window]
      print('buffer_sw is {}, buffer is {}, sw is {}'.format(
          buffer[skip_window], buffer, skip_window))
#     labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    print(data_index)
    data_index = (data_index + 1) % len(data)

3
buffer_sw is 2, buffer is deque([4, 2, 6], maxlen=3), sw is 1
buffer_sw is 2, buffer is deque([4, 2, 6], maxlen=3), sw is 1
10
buffer_sw is 6, buffer is deque([2, 6, 1], maxlen=3), sw is 1
buffer_sw is 6, buffer is deque([2, 6, 1], maxlen=3), sw is 1
11
buffer_sw is 1, buffer is deque([6, 1, 8], maxlen=3), sw is 1
buffer_sw is 1, buffer is deque([6, 1, 8], maxlen=3), sw is 1
12
buffer_sw is 8, buffer is deque([1, 8, 9], maxlen=3), sw is 1
buffer_sw is 8, buffer is deque([1, 8, 9], maxlen=3), sw is 1
13


In [12]:
print(labels)

[[6]
 [6]
 [1]
 [1]
 [8]
 [8]
 [2]
 [2]]


In [13]:
print(batch)

[2 2 6 6 1 1 8 8]


In [14]:
t = 1
t2av = [t]
while t in t2av:
    t = random.randint(0, span - 1)    
t2av.append(t)
print(t, t2av)

2 [1, 2]


In [15]:
random.randint(0, 3)

2

In [16]:
sdf

NameError: name 'sdf' is not defined

In [ ]:
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):  # _ 表示对循环中的实际值我们没有兴趣，为的是把 buffer 填充满
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
      '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

In [ ]:
words

In [ ]:
reverse_dictionary

In [ ]:
labels[:, 0]

In [ ]:
reverse_dictionary[batch[0]]

In [ ]:
batch[1]

In [ ]:
words

In [ ]:
batch

In [1]:
import tensorflow as tf

In [2]:
vocabulary_size = 10
embedding_size = 4

In [4]:
embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

In [8]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    init.run()
    print(sess.run(embeddings))

[[ 0.07638097 -0.74863482  0.64941716  0.96933889]
 [ 0.99118352  0.15787339  0.51375771  0.73684525]
 [ 0.64092541  0.90945625 -0.29194331  0.81878996]
 [ 0.52478266  0.10064077  0.77446246 -0.16088438]
 [ 0.50437546 -0.35315323  0.12309933  0.04595327]
 [-0.52837586 -0.75811768  0.79396987 -0.43218112]
 [-0.42022991 -0.97509933  0.78607631 -0.02260661]
 [ 0.33184648 -0.41429234  0.80781531  0.78594446]
 [-0.39726686  0.10750365  0.01810956 -0.15397811]
 [-0.08630228  0.2818079  -0.38067865 -0.39891505]]
